# Segmenting and Clustering Neighborhoods in Toronto

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0         conda-forge
    geopy:         1.20.0-py_0       conda-forge

The following p

## Scraping wikipedia page to extract Neighborhodds in Toronto

In [3]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [4]:
!pip install beautifulsoup4
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'html.parser')


     |████████████████████████████████| 102kB 11.2MB/s ta 0:00:01


In [5]:
My_table = soup.find_all('table', class_='sortable')
#My_table

In [6]:
for table in My_table:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    if headings[:3] == ['postcode', 'borough', 'neighbourhood']:
        break

# Extract the columns we want and write to a semicolon-delimited text file.
with open('Toronto_neighbourhoods.txt', 'w') as fo:
    for tr in table.find_all('tr'):
        tds = tr.find_all('td')
        if not tds:
            continue
        postcode, borough, neighbourhood = [td.text.strip() for td in tds[:3]]
        # Wikipedia does something funny with country names containing
        # accented characters: extract the correct string form.
        print('; '.join([postcode, borough, neighbourhood]), file=fo)


In [37]:
#Create a dataframe from the extracted txt file
df = pd.read_csv('Toronto_neighbourhoods.txt', sep=";", header=None)
df.columns=['Postal Code', 'Borough', 'Neighborhood']
df.shape

(288, 3)

## Removing the entries which are Not assigned

In [84]:
df=df.drop(df[df['Borough']==' Not assigned'].index)
df.shape

(211, 3)

## Merging multiple neighborhood with similar postal codes

In [100]:
df1 = df.groupby('Postal Code').agg({'Borough':'first','Neighborhood': ', '.join})
df3=df[{'Postal Code'}]
df4=df3.drop_duplicates()
df_inner = pd.merge(df4, df1, on='Postal Code', how='inner')


### Reassigning Neighborhood name to Borough name if it is not assigned

In [106]:
df_inner=df_inner.replace(to_replace=r' Not assigned', value='Queen\'s Park', regex=True)

In [108]:
df_inner.shape

(103, 3)

## Adding Geographical Co-ordinates of the Neighborhoods

In [105]:
df_loc = pd.read_csv('https://cocl.us/Geospatial_data')

In [109]:
df_loc.shape

(103, 3)

## Combining both dataframes

In [110]:
df_final = pd.merge(df_inner, df_loc, on='Postal Code', how='inner')

df_final.shape


(103, 5)

In [111]:
df_final

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


### cluster the neighborhoods in Toronto with only boroughs that contain the word Toronto

In [136]:
toronto_data = df_final[df_final['Borough'].str.contains(' Toronto', regex=False)].reset_index(drop=True)
toronto_data.shape

(38, 5)

In [137]:
toronto_data

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
8,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259
9,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union S...",43.640816,-79.381752


## Let's get the geographical coordinates of Toronto

In [138]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


## Visualize Toronto and neighborhoods in it

In [139]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Cluster the Toronto Neighborhoods using K-Means Clustering Algorithm

In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 